In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')

# visualize results module
from visualize import visualize_svm as Visualize

import scipy
from tqdm import tqdm

Autosaving every 180 seconds


<ipython-input-1-9bc938cb63d7>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
###########################################
########### HELPER FUNCTIONS ##############
###########################################
def load_reward_times(root_dir,
                      animal_id,
                      sessions,
                      width):
    # load rewarded and nonrewarded times in miliseconds
    rtimes = np.loadtxt(os.path.join(root_dir,
                                  animal_id,
                                  'tif_files',
                                  sessions[0],
                                  'rewarded_times.txt'))*1E3

    #
    nonrtimes = np.loadtxt(os.path.join(root_dir,
                                  animal_id,
                                  'tif_files',
                                  sessions[0],
                                  'nonrewarded_times.txt'))*1E3

    # add all data togheter
    # rtimes = np.concatenate((rtimes, nonrtimes))

    # 
    rtimes_random = []
    for k in range(rtimes.shape[0]):
        temp = rtimes[k] 
        rr = (np.random.rand(1)*50-25)*1E3
        #print (temp, rr)
        rtimes_random.append(temp+rr)

    # convolve all data with gaussian

    start = -10*1E3
    end = 10*1E3
    x = np.arange(start, end, 1) # entire range of x, both in and out of spec
    std = 1.5*1E3  # std in miliseconds
    mu = 0
    y = scipy.stats.norm.pdf(x,mu,std) 

    t = np.zeros(int(1400*1E3))
    for r in tqdm(rtimes):
        try:
            t[int(r+start):int(r+end)]+=y
        except:
            pass

    t_random = np.zeros(int(1400*1E3))
    for r in tqdm(rtimes_random):
        try:
            t_random[int(r+start):int(r+end)]+=y
        except:
            pass

    #####################################
    # stack segments on top of each other
    stack = []
    stack_random = []
    for r in tqdm(rtimes):
        temp = t[int(r-width):int(r+width)]
        if temp.shape[0]==(2*width):
            stack.append(temp)

        temp = t_random[int(r-width):int(r+width)]
        if temp.shape[0]==(2*width):
            stack_random.append(temp)


    stack = np.vstack(stack).T
    stack_random = np.vstack(stack_random).T
    print ("stacK: ", stack.shape)

    ######################################
    # sample back down to 30Hz
    t = t[::30]
    x = np.arange(t.shape[0])/30.
    
    return x, t, stack, stack_random

#
def plot_n_trials_per_session(root_dir,
                             animal_id,
                             session_id):

    arr = []
    ctr=0
    for animal_id in animal_ids:
        sessions = Visualize.get_sessions(root_dir,
                                          animal_id,
                                          session_id)


        # load rewarded and nonrewarded times in miliseconds
        rtimes = np.loadtxt(os.path.join(root_dir,
                                      animal_id,
                                      'tif_files',
                                      sessions[0],
                                      'rewarded_times.txt'))

        # load rewarded and nonrewarded times in miliseconds
        n_trials = []
        for session in tqdm(sessions):
            try:
                import warnings
                with warnings.catch_warnings():
                     warnings.simplefilter("ignore")
                     #data = np.loadtxt(myfile, unpack=True)
                     rtimes = np.loadtxt(os.path.join(root_dir,
                                          animal_id,
                                          'tif_files',
                                          session,
                                          'rewarded_times.txt'), unpack=True)
                if rtimes.shape[0]>30:
                    n_trials.append(rtimes.shape[0])
            except:
                pass
        n_trials = np.array(n_trials)
        idx = np.where(np.isnan(n_trials)==False)[0]
        n_trials = n_trials[idx]

        arr.append(n_trials)
        ctr+=1
        #if ctr>1:
        #    break

    #####################################
    linestyles = ['-','-','-']
    pvals = [0.05,0.01,0.001,0.0001,0.00001]
    my_dict = dict(
                    M1 = arr[0], 
                    M2 = arr[1], 
                    M3 = arr[2], 
                    M4 = arr[3], 
                    M5 = arr[4], 
                    M6 = arr[5],                
                  )

    # 
    #cons.append(edts[1])
    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    ###############################################
    ########## PLOT SCATTER #######################
    ###############################################
    fig = plt.figure(figsize=(24,8))
    clrs_local = ["black","black","black","black","black","black",] #,"red"]
    for i,d in enumerate(data):
        y = data[d]
        #print ('y: ', y)
        x = np.random.normal(i+1, 0.04, len(y))
        if True:
            plt.plot(x, y, 
                 mfc =clrs_local[i], 
                 mec='k', 
                 ms=7, 
                 marker="o", 
                 linestyle="None",
                 alpha=.8
                )
        else:
            plt.scatter(x, y, 
                   c=clrs_local[i],
                   edgecolor='black',
                   s=200,
                   alpha=.7)


    ###############################################
    ########## PLOT  #######################
    ###############################################
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    bplot = data.boxplot(showfliers=True,
                flierprops=flierprops,
                grid=False) 

    plt.ylabel("# of trials in session")

    #
    plt.ylim(29,200)  
    
#   
def plot_average_convolution(width, stack, stack_random):

    plt.figure(figsize=(8,8))
    t= np.arange(-width,width,1)/1E3
    plt.plot(t,stack,c='blue',alpha=.01)
    # plt.plot(t,stack[:,0],c='black',alpha=0, label='singletrial')
    signal = stack.mean(1)
    plt.plot(t,signal,
             c='darkblue',
             linewidth=3,
             label="Session average")
    #
    if True:
        signal_random = stack_random.mean(1)
        plt.plot(t,signal_random,
                 c='darkred',
                 linewidth=3,
                 alpha=.7,
                 label="Session average - random")


        
    plt.plot([-3,-3],
             [0,0.00030],'--',
            c='black')
    plt.plot([3,3],
             [0,0.00030],'--',
            c='black')
    plt.xlim(t[0],t[-1])
    xticks=[-50,-40,-30,-20,-10,-3,0,
            3,10,20,30,40,50]
    plt.xticks(xticks)
    plt.xlabel("Time (sec)")
    plt.yticks([])

    plt.title(sessions[0]+ " # of trials: "+ str(stack.shape[1]))
    
    
    
    
#    
# def plot_average_convolution_hilbert(width, stack, stack_random):

#     # this function works on the simulated neural signals; so thereare no ROIs
    
#     plt.figure(figsize=(8,8))
#     print ("stack: ", stack.shape)
#     mean = stack.mean(1)
#     print ("means: ", mean.shape)
    
#     #
#     analytic_signal = hilbert(mean)
#     amplitude_envelope = np.abs(analytic_signal)
#     #amplitude_envelope = butter_lowpass_filter(amplitude_envelope,.5,30)

#     start = int(0*1E3)
#     end = int(1050*1E3)
    
#     # 
#     t = mean[start:end]
#     x = np.arange(t.shape[0])/1E3
#     amplitude_envelope = amplitude_envelope[start:end]

#     #
#     plt.plot(x, t, c='darkblue', linewidth=5, label='Average session neural signal')
#     plt.plot(x, amplitude_envelope, '--', c='black', linewidth=1, label="Hilbert transform envelope")


#     #
#     # plt.plot([x[0], x[-1]],[0,0],'--',c='black', linewidth=3)
#     # plt.plot([x[600], x[600]],[-5,7],'--',c='black', linewidth=3)

    
#     #plt.plot(tt,means[:,feature_id],c='blue')
#     #plt.plot(t,random.mean(0)[:,4],c='red')
#     #plt.xlim(-30,3)
#     plt.title(animal_id+" " +str(lockout)+"sec")
#     #ctr+=1
    
#     #xticks=[-50,-40,-30,-20,-10,-3,0,
#     #        3,10,20,30,40,50]
#     #plt.xticks(xticks)
#     #plt.xlim(t[0],t[-1])
    
    
# #     plt.plot([-3,-3],
# #              [0,0.00030],'--',
# #             c='black')
# #     plt.plot([3,3],
# #              [0,0.00030],'--',
# #             c='black')
   
#     plt.xlabel("Time (sec)")
#     plt.yticks([])

#     plt.title(sessions[0]+ " # of trials: "+ str(stack.shape[1]))

In [13]:
###########################################
############ LOAD LEVER PULL TIMES ########
###########################################
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'
session_id = 'IA1pm_Feb2_30Hz'
session_id = 'IA1pm_Feb16_30Hz'
session_id = 'IA1pm_Feb3_30Hz'
session_id = 'IA1pm_Feb9_30Hz'

# animal_id = 'IA2'
# session_id = 'IA2pm_Mar14_30Hz'
# session_id = 'IA2pm_Feb29_30Hz'
# session_id = 'IA2pm_Feb3_30Hz'

# animal_id = 'IA3'
# session_id = 'IA3pm_Apr1_30Hz'
# session_id = 'IA3pm_Feb4_30Hz'


animal_id = 'IJ1'
# # session_id = 'IJ1am_Mar10_30Hz'
# #session_id = 'IJ1pm_Mar2_30Hz'
# session_id = 'IJ1pm_Mar3_30Hz'
session_id = 'IJ1pm_Feb11_30Hz'

# animal_id = 'IJ2'
# session_id = 'IJ2pm_Apr5_30Hz'
# session_id = 'IJ2pm_Feb9_30Hz'

#animal_id = 'AQ2'
# session_id = 'AQ2am_Apr1_Week4_30Hz'
#session_id = 'AQ2am_Dec30_30Hz'
# session_id = 'AQ2am_Apr7_Week5_30Hz'
# session_id = 'AQ2am_Apr1_Week4_30Hz'

# session_id = 'all'
# session = session_id  # or 'all'
sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  session_id)

# sessions = [sessions[2]]
print ("# sessions: ", sessions)


#
width= 50*1E3  # in milliseconds

x, t, stack, stack_random = load_reward_times(root_dir,
                                               animal_id,
                                               sessions,
                                               width)



100%|██████████| 49/49 [00:00<00:00, 164944.54it/s]

# sessions:  ['IJ1pm_Feb11_30Hz']
stacK:  (100000, 48)


In [5]:
#####################################################################
####### PLOT CONVOLUTION (LEVER PULL, GAUSSIAN) ACROSS SESSION ######
#####################################################################
plt.figure(figsize=(24,8))
plt.plot(x,t)
plt.xlabel("Time (sec)")
plt.ylabel("Convolution (lever pull time, gaussian)")
plt.yticks([])
plt.xlim(x[0],x[-1])
if False:
    plt.savefig('/home/cat/rtimesgaussian.svg')
    plt.close()
else:
    plt.show()

In [14]:
#######################################
####### PLOT AVERAGE CONVOLUTION ######
####################################### 
#     
plot_average_convolution(width,
                         stack,
                         stack_random)
plt.xlim(-20,5)
if False:
    plt.savefig('/home/cat/stacks.svg')
    plt.close()
else:
    plt.show()



In [16]:
# ################################################
# ####### PLOT AVERAGE NUMBER OF LEVER PULLS #####
# ################################################
# # plot # of lever pulls per session per animal as scatter plot
# import pandas as pd
# root_dir = '/media/cat/4TBSSD/yuki/'

# animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
# session_id = 'all'

# plot_n_trials_per_session(root_dir,
#                          animal_id,
#                          session_id)

# #
# if False:
#     plt.savefig('/home/cat/n_trials_per_session.svg')
#     plt.close()
# else:
#     plt.show()

In [17]:
########################################################
#### PLOT READINESS POTENTIAL FOR LOCKED OUT DATA ######
########################################################
from scipy.signal import butter, lfilter, filtfilt, hilbert, chirp

def butter_lowpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = butter(order, normal_cutoff, btype='lowpass')
    return b, a

def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=4):
    # Source: https://github.com/guillaume-chevalier/filtering-stft-and-laplace-transform
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = filtfilt(b, a, data)
    return y



animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_ids = ['IJ2']

clrs = ['darkblue','red']
lockouts = [0,21]  #

max_trials_in_animal = [399,   # 21 sec min times
                        385,
                        96,
                        137,
                        82,
                        318]


# max_trials_in_animal = [601,   # 12 sec min trials
#                         728,
#                         310,
#                         371,
#                         197,
#                         910]

#
plt.figure(figsize=(20,4))
ctr=0

# 
for animal_id in animal_ids:
    ax=plt.subplot(1,6,ctr+1)
    
    for ctr_color, lockout in enumerate(lockouts): 

        data = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz',
                       allow_pickle=True)

        names = data['names']
        behaviors = data['behaviors']
        trials = data['trials']

        temp = []
        for tr in trials:
            if tr.shape[0]<=30:
                temp.append(tr)
        
            
            
        trials = np.vstack(temp)
        print ("trials: ", trials.shape)
        random = data['random']
        random = np.vstack(random)
        


        #
        if False:
            maxes = max_trials_in_animal[ctr]
            #n_tr = min(trials.shape[0],300)
            idx = np.random.choice(np.arange(trials.shape[0]),maxes, replace=False)
        else:
            idx = np.arange(trials.shape[0])
        
        means = trials[idx].mean(0)
        tt= np.arange(trials.shape[1])/30-30

        #
        feature_id = 4

        #
        x = tt.copy()

        #
        # analytic_signal = hilbert(means[:,feature_id])
        # amplitude_envelope = np.abs(analytic_signal)
        # amplitude_envelope = butter_lowpass_filter(amplitude_envelope,.5,30)

        start = 0
        end = 1050
        t = means[:,feature_id][start:end]
        
        #t = butter_lowpass_filter(t,1,30)
        
        t = t/np.max(t)

        # 
        x = x[start:end]
        #amplitude_envelope = amplitude_envelope[start:end]

        plt.plot(x, t, c=clrs[ctr_color], 
                 linewidth=5, 
                 alpha=0.6,
                 label='Average session neural signal')
        # plt.plot(x, amplitude_envelope, '--', c='black', linewidth=1, label="Hilbert transform envelope")


        #
        plt.plot([x[0], x[-1]],[0,0],'--',c='black', linewidth=3)
        plt.plot([x[900], x[900]],[-1,1],'--',c='black', linewidth=3)


        #plt.plot(tt,means[:,feature_id],c='blue')
        #plt.plot(t,random.mean(0)[:,4],c='red')
        plt.xlim(-20,3)
        #plt.title(animal_id+" " +str(lockout)+"sec")
        plt.yticks([])
        plt.xticks([])
    ctr+=1
    
if False:
    plt.savefig('/home/cat/allaverages_'+str(lockouts)+'.svg')
    plt.close()
else:
    plt.show()


trials:  (165, 1800, 16)
trials:  (82, 1800, 16)


In [10]:
# data = np.load('/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz',
#                allow_pickle=True)

# behaviors = data['all_area_names']
# for ctr,b in enumerate(behaviors[0]):
#     print (ctr, b)


0 Retrosplenial area, dorsal part, layer 1
1 Retrosplenial area, lateral agranular part, layer 1
2 Primary visual area, layer 1
3 Primary somatosensory area, upper limb, layer 1
4 Primary somatosensory area, lower limb, layer 1
5 Primary somatosensory area, barrel field, layer 1
6 Secondary motor area, layer 1
7 Primary motor area, Layer 1
8 Primary motor area, Layer 1
9 Secondary motor area, layer 1
10 Primary somatosensory area, barrel field, layer 1
11 Primary somatosensory area, lower limb, layer 1
12 Primary somatosensory area, upper limb, layer 1
13 Primary visual area, layer 1
14 Retrosplenial area, lateral agranular part, layer 1
15 Retrosplenial area, dorsal part, layer 1


In [11]:
# if True:
#     d1 = np.load('/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz',
#                  allow_pickle=True)
#     trials = d1['trials']
#     print (trials.shape)

#     sessions2 = d1['all_session_names']
#     #print (sessions2)
#     for k in range(len(sessions2)):
#         if "Feb" in sessions2[k]:
#             print (k,sessions2[k], trials[k].shape)    
 

(41,)
10 IA3pm_Feb1_30Hz (30, 1800, 16)
11 IA3pm_Feb2_30Hz (65, 1800, 16)
12 IA3pm_Feb3_30Hz (94, 1800, 16)
13 IA3pm_Feb4_30Hz (54, 1800, 16)
14 IA3pm_Feb9_30Hz (11, 1800, 16)
15 IA3pm_Feb11_30Hz (13, 1800, 16)
16 IA3pm_Feb12_30Hz (83, 1800, 16)
17 IA3pm_Feb15_30Hz (48, 1800, 16)
18 IA3pm_Feb16_30Hz (66, 1800, 16)
19 IA3pm_Feb17_30Hz (50, 1800, 16)
20 IA3pm_Feb18_30Hz (38, 1800, 16)
21 IA3pm_Feb19_30Hz (30, 1800, 16)
22 IA3pm_Feb22_30Hz (24, 1800, 16)
23 IA3pm_Feb23_30Hz (11, 1800, 16)
24 IA3pm_Feb26_30Hz (67, 1800, 16)
25 IA3pm_Feb29_30Hz (49, 1800, 16)


In [18]:
#############################################
#############################################
#############################################
session_id = 13
feature_id = 4
    
#
print (trials[session_id].shape)
means = trials[session_id].mean(0)

plt.figure(figsize=(8,8))
plt.plot([-3,-3],[-30,100],'--',c='black')
plt.plot([3,3],[-30,100],'--',c='black')
t=np.arange(means.shape[0])/30-30
plt.plot(t,means[:,feature_id])
plt.title(sessions2[session_id])
plt.xlim(-20,5)

if False:
    plt.savefig('/home/cat/example_oscillations.svg')
    plt.close()
else:
    plt.show()


(1800, 16)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed